In [ ]:
import tensorflow as tf
from tensorflow import keras
from glob import glob
import os
import numpy as np
import random

# set bsize, num val images and list files

In [ ]:
class_map = {"acf_alv":0, "acf_chap":1, "reboco":2, "tijolo_alv":3, "tijolo_chap":4}

BATCH_SIZE = 64


train_images_path = glob("resized_dataset_texturas_v2/*/*.jpeg")
random.shuffle(train_images_path)
val_images_path = glob("resized_dataset_texturas_v2_test/*/*.jpeg")
random.shuffle(val_images_path)

train_files = train_images_path
val_files = val_images_path

def filename_to_label(file_names, class_map_, class_index=1):
    labels = []
    for file_name in file_names:
        labels.append(class_map_[file_name.split("/")[class_index]])
    return labels

train_labels = filename_to_label(train_files, class_map)
val_labels = filename_to_label(val_files, class_map)


# define data handling functions

In [ ]:
def read_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image.set_shape([256, 256, 3])    
    return image
    

def load_data_for_autoencoder(image_list):
    image = read_image(image_list)    
    image = tf.cast(image, tf.float16) / 256    
    return image, image

def load_data_for_classifier(image_list, label_list, oh_depth):
    image = read_image(image_list)    
    image = tf.cast(image, tf.float16) / 256    
    label = tf.one_hot(label_list, oh_depth, 1.0, 0.0)
    label.set_shape([5])
    return image, label


def autoencoder_data_generator(image_list, drop_remainder=True):
    dataset = tf.data.Dataset.from_tensor_slices((image_list))
    dataset = dataset.shuffle(2048)        
    dataset = dataset.map(load_data_for_autoencoder, num_parallel_calls=tf.data.AUTOTUNE)    
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=drop_remainder, num_parallel_calls=tf.data.AUTOTUNE)    
    return dataset

def classifier_data_generator(image_list, labels_list, oh_depth, drop_remainder=True):
    dataset = tf.data.Dataset.from_tensor_slices((image_list, labels_list, oh_depth))
    dataset = dataset.shuffle(2048)        
    dataset = dataset.map(load_data_for_classifier, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=drop_remainder, num_parallel_calls=tf.data.AUTOTUNE)    
    return dataset


train_dataset = autoencoder_data_generator(train_files)
val_dataset = autoencoder_data_generator(val_files, False)

print("Train Dataset:", train_dataset)
print("Val Dataset:", val_dataset)


# define loss functions

In [ ]:
def custom_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    return tf.reduce_mean(tf.abs(tf.subtract(y_true, y_pred)))

# creating classifier

In [ ]:
autoencoder_model = tf.keras.models.load_model(
    r"./saved_models/encoder_v3",
    custom_objects={"custom_loss": None},
)

autoencoder_model.trainable = False
model_input = autoencoder_model.layers[0].input
encoder_output = autoencoder_model.get_layer("encoder_output").output
encoder = keras.Model(inputs=[model_input], outputs=[encoder_output])

x = x_in = keras.layers.Input((256,256,3))

encoder_output = encoder(x)


x = keras.layers.Flatten()(encoder_output)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Dense(5, "softmax")(x)

# create model
classifier = keras.Model(inputs=[x_in], outputs=[x])

classifier.summary()

# load classifier data

In [ ]:
train_dataset = classifier_data_generator(
    train_files, train_labels, oh_depth=tf.constant([5] * len(train_labels))
)
val_dataset = classifier_data_generator(
    val_files,
    val_labels,
    oh_depth=tf.constant([5] * len(val_labels)),
    drop_remainder=False,
)

print("Train Dataset:", train_dataset)
print("Val Dataset:", val_dataset)

# train classifier

In [ ]:

lr_inicial = 0.00001
lr_default = 0.001
lr_final = 0.00001
epochs = 100

# This function warmup the learning rate in the first 
def schedulerWithWarmupAndDecay(epoch, lr):
    if epoch < int(epochs * 0.1):
        return lr + (lr_default / (epochs * 0.1 + 1))
    elif epoch > int(epochs * 0.60):
        return lr_final
    else:
        return lr_default

# callback for updating the learning rate
lr_callback = tf.keras.callbacks.LearningRateScheduler(schedulerWithWarmupAndDecay)

opt = keras.optimizers.Adam(lr_inicial)
classifier.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=["accuracy"])
history = classifier.fit(train_dataset, epochs=epochs, callbacks=[lr_callback])

# save classifier
classifier.save("saved_models/classifier_v2/", overwrite=False)
classifier.save("saved_models/classifier_v2.h5", overwrite=False)

# load classifier

In [ ]:
classifier = tf.keras.models.load_model(
    r"./saved_models/classifier_v2"
)

# evaluate classifier

In [ ]:
classifier.evaluate(val_dataset)

# visualize classifier predictions

In [ ]:
import matplotlib.pyplot as plt

class_name_map = ["acf_alv", "acf_chap", "reboco", "tijolo_alv", "tijolo_chap"]

examples = val_dataset.take(1)
for e in examples.as_numpy_iterator():
    imgs = e[0]
    lbls = e[1]
    for img, lbl in zip(imgs, lbls):
        print(lbl.shape)
        prediction = classifier.predict(img[tf.newaxis, ...])[0]
        classifier_prediciton = class_name_map[np.argmax(prediction)]
        
        plt.imshow(img.astype(np.float32))
        plt.show()

        
        print(classifier_prediciton)

        print("-------------------------\n\n")
    break

# save classifier predictions

In [ ]:
import matplotlib.pyplot as plt
import cv2 as cv

class_name_map = ["acf_alv", "acf_chap", "reboco", "tijolo_alv", "tijolo_chap"]

save_path = "classifier_predictions/"

examples = train_dataset.take(-1)
i = 0
for e in examples.as_numpy_iterator():
    imgs = e[0]
    lbls = e[1]
    for img, lbl in zip(imgs, lbls):
        print(lbl.shape)
        prediction = classifier.predict(img[tf.newaxis, ...])[0]
        classifier_prediciton = class_name_map[np.argmax(prediction)]
        write_image_path = save_path + classifier_prediciton

        # plt.imshow(img.astype(np.float32))
        # plt.show()

        # Create the output folder if it doesn't exist
        if not os.path.exists(write_image_path):
            os.makedirs(write_image_path)
        
        img = (img*255).astype(np.uint8)        
        img = cv.cvtColor(img, cv.COLOR_RGB2BGR)

        cv.imwrite(write_image_path + f"/{class_name_map[np.argmax(lbl)]}_{i}.jpeg", img)

        print(classifier_prediciton)
        print("-------------------------\n\n")
        i += 1


examples = val_dataset.take(-1)
for e in examples.as_numpy_iterator():
    imgs = e[0]
    lbls = e[1]
    for img, lbl in zip(imgs, lbls):
        print(lbl.shape)
        prediction = classifier.predict(img[tf.newaxis, ...])[0]
        classifier_prediciton = class_name_map[np.argmax(prediction)]
        write_image_path = save_path + classifier_prediciton

        # plt.imshow(img.astype(np.float32))
        # plt.show()

        # Create the output folder if it doesn't exist
        if not os.path.exists(write_image_path):
            os.makedirs(write_image_path)
        
        img = (img*255).astype(np.uint8)        
        img = cv.cvtColor(img, cv.COLOR_RGB2BGR)

        cv.imwrite(write_image_path + f"/{class_name_map[np.argmax(lbl)]}_{i}.jpeg", img)

        print(classifier_prediciton)
        print("-------------------------\n\n")
        i += 1
    

# Generate Confusion Matrix

In [ ]:


# Extract true labels from the dataset
true_labels = []
imgs = []
for _, label in val_dataset:
    imgs.append(_)
    true_labels.append(label.numpy())

true_labels = np.concatenate(true_labels)
true_labels = np.argmax(true_labels, axis=-1)
print(true_labels)

predicted_labels = classifier.predict(imgs)
predicted_labels = np.argmax(predicted_labels, axis=-1)
print(predicted_labels)


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import numpy as np
from keras.models import load_model


# Compute confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=class_name_map)
disp.plot()
plt.show()
print("Confusion Matrix:")
print(cm)
